In [1]:
! pip3 install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.3 MB/s eta 0:00:00


In [1]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

#Pre trained tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

#Pre trained Model for Serquence Classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv',encoding='latin-1')
# df.head()
texts = df['text'][:3].tolist()
sentiments = df['sentiment'][:3].tolist()
# len(texts)

In [59]:
# df.head()

In [60]:
for i in range(len(texts)):
  texts[i]=str(texts[i])

In [61]:
encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')


In [62]:
input_ids = encoded_texts['input_ids']
attention = encoded_texts['attention_mask']

In [63]:
sentiment=[]
for s in sentiments:
  if s=='negative':
    sentiment.append(0)
  elif s=='neutral':
    sentiment.append(1)
  else:
    sentiment.append(2)


In [64]:
import torch.nn as nn

# Add a custom classification head on top of the pre-trained model
num_classes = len(set(sentiment))
classification_head = nn.Linear(model.config.hidden_size, num_classes)

# Replace the pre-trained model's classification head with our custom head
model.classifier = classification_head

In [66]:
import torch.optim as optim
import torch
# Define the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()
model.train()

# Fine-tune the model
num_epochs = 30
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(input_ids, attention_mask= attention, labels=torch.tensor(sentiment))
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/30, Loss: 0.5088891386985779
Epoch 2/30, Loss: 0.5050331354141235
Epoch 3/30, Loss: 0.4660484790802002
Epoch 4/30, Loss: 0.36847224831581116
Epoch 5/30, Loss: 0.38147369027137756
Epoch 6/30, Loss: 0.34678909182548523
Epoch 7/30, Loss: 0.24565644562244415
Epoch 8/30, Loss: 0.2274204045534134
Epoch 9/30, Loss: 0.21354730427265167
Epoch 10/30, Loss: 0.19727979600429535
Epoch 11/30, Loss: 0.1668061763048172
Epoch 12/30, Loss: 0.1315010040998459
Epoch 13/30, Loss: 0.11128357797861099
Epoch 14/30, Loss: 0.10761389881372452
Epoch 15/30, Loss: 0.08985219150781631
Epoch 16/30, Loss: 0.0823407769203186
Epoch 17/30, Loss: 0.07459253072738647
Epoch 18/30, Loss: 0.0769784078001976
Epoch 19/30, Loss: 0.07080280780792236
Epoch 20/30, Loss: 0.05922015383839607
Epoch 21/30, Loss: 0.055901508778333664
Epoch 22/30, Loss: 0.05787165090441704
Epoch 23/30, Loss: 0.04901997745037079
Epoch 24/30, Loss: 0.054262518882751465
Epoch 25/30, Loss: 0.04326346144080162
Epoch 26/30, Loss: 0.04066099599003792
E

In [69]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv',encoding='latin-1')
df.head()

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0


In [79]:
texts = df['text'][:300].tolist()
sentiments = df['sentiment'][:300].tolist()
for i in range(len(texts)):
  texts[i]=str(texts[i])
encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
# input_ids = encoded_texts['input_ids']
# attention = encoded_texts['attention_mask']
sentiment=[]
for s in sentiments:
  if s=='negative':
    sentiment.append(0)
  elif s=='neutral':
    sentiment.append(1)
  else:
    sentiment.append(2)


In [80]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_outputs = model(**encoded_texts)
    predictions = torch.argmax(test_outputs.logits, dim=1).tolist()

In [82]:
from sklearn.metrics import accuracy_score

# Assuming 'labels' is the column in your test.csv file containing the true labels
# true_labels = test_df['labels'].tolist()

# Calculate accuracy
accuracy = accuracy_score(sentiment, predictions)

# Print the accuracy
print(f'Accuracy: {accuracy * 100:.2f}%')